In [44]:
import os
from steam_web_api import Steam

KEY = os.environ.get("STEAM_API_KEY")
steam = Steam(KEY)

terraria_app_id = 105600

# arguments: app_id
user = steam.apps.get_app_details(terraria_app_id)
print(user)

{'105600': {'success': True, 'data': {'type': 'game', 'name': 'Terraria', 'steam_appid': 105600, 'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': [409210, 1323320], 'detailed_description': 'Dig, Fight, Explore, Build:  The very world is at your fingertips as you fight for survival, fortune, and glory.   Will you delve deep into cavernous expanses in search of treasure and raw materials with which to craft ever-evolving gear, machinery, and aesthetics?   Perhaps you will choose instead to seek out ever-greater foes to test your mettle in combat?   Maybe you will decide to construct your own city to house the host of mysterious allies you may encounter along your travels? <br><br>In the World of Terraria, the choice is yours!<br><br>Blending elements of classic action games with the freedom of sandbox-style creativity, Terraria is a unique gaming experience where both the journey and the destination are completely in the player’s control.   The Terraria adventure

In [45]:
import sqlite3
import requests
import tweepy
import os
import json
from dotenv import load_dotenv
import tempfile
from github import Github
import base64
import time
from datetime import datetime, timedelta
import pytz


In [46]:
# Charger les variables d'environnement
load_dotenv()

True

In [47]:
# Configuration
TWEETED_GAMES_FILE = 'tweeted_games.json'
GITHUB_REPO = 'newgame-newtweet'
GITHUB_FILE_PATH = 'tweeted_games.json'
PARIS_TZ = pytz.timezone('Europe/Paris')
TWEET_START_TIME = PARIS_TZ.localize(datetime.now().replace(hour=11, minute=0, second=0, microsecond=0))
TWEET_INTERVAL = timedelta(minutes=10)
TIMESTAMP_THRESHOLD = 1721681830 #pour faire démarrer les tweets à partir d'environ le 22/07, date de début de ce projet

In [58]:
#pour télécharger steam_games.db de manière temporaire 
def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

#pour lire le contenu du fichier "tweeted_games"
def get_github_file_content(file_path, decode=False):
    url = f"https://api.github.com/repos/{os.getenv('GITHUB_USERNAME')}/{GITHUB_REPO}/contents/{file_path}"
    headers = {'Authorization': f"token {os.getenv('GITHUB_TOKEN')}"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        content = base64.b64decode(response.json()['content'])
        if decode:
            try:
                return json.loads(content.decode('utf-8'))
            except json.JSONDecodeError:
                print(f"Le fichier {file_path} est vide ou n'est pas un JSON valide.")
                return []
        return content
    elif response.status_code == 404:
        print(f"Le fichier {file_path} n'existe pas.")
        return []
    else:
        print(f"Erreur lors de la récupération du fichier: {response.status_code}")
        return None


#pour actualiser le contenu du fichier "tweeted_games"
def update_github_file(content, commit_message):
    g = Github(os.getenv('GITHUB_TOKEN'))
    repo = g.get_repo(f"{os.getenv('GITHUB_USERNAME')}/{GITHUB_REPO}")
    try:
        contents = repo.get_contents(GITHUB_FILE_PATH)
        repo.update_file(GITHUB_FILE_PATH, commit_message, json.dumps(content), contents.sha)
        print(f"Fichier {GITHUB_FILE_PATH} mis à jour avec succès.")
    except Exception as e:
        if "Not Found" in str(e):
            repo.create_file(GITHUB_FILE_PATH, commit_message, json.dumps(content))
            print(f"Fichier {GITHUB_FILE_PATH} créé avec succès.")
        else:
            print(f"Erreur lors de la mise à jour du fichier: {str(e)}")

def load_tweeted_games():
    content = get_github_file_content(GITHUB_FILE_PATH, decode=True)
    return set(content) if content is not None else set()

def save_tweeted_games(tweeted_games):
    update_github_file(list(tweeted_games), "Mise à jour des jeux tweetés")

def check_new_entries(conn, tweeted_games):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT app_id 
        FROM games 
        WHERE first_seen >= ? AND app_id NOT IN (?)
    """, (TIMESTAMP_THRESHOLD, ','.join(map(str, tweeted_games))))
    return [app_id for (app_id,) in cursor.fetchall()]

def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]['success']:
            return data[str(app_id)]['data']
    return None

def filter_game(game_data):
    return game_data['type'] == 'game' and not game_data.get('dlc', False)

def get_twitter_client():
    client = tweepy.Client(
        bearer_token=os.getenv('TWITTER_BEARER_TOKEN'),
        consumer_key=os.getenv('TWITTER_CONSUMER_KEY'),
        consumer_secret=os.getenv('TWITTER_CONSUMER_SECRET'),
        access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
        access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
    )
    return client

def create_tweet_message(game_data):
    name = game_data['name']
    genres = ", ".join([genre['description'] for genre in game_data['genres']])
    description = game_data['short_description']
    developers = ", ".join(game_data['developers'])
    return f"{name}. {genres}. \"{description}\". Développé par: {developers}"

def tweet_message(message):
    client = get_twitter_client()
    try:
        response = client.create_tweet(text=message)
        return response.data['id']
    except Exception as e:
        print(f"Erreur lors de la création du tweet: {e}")
        return None

def wait_until(time_to_wait):
    now = datetime.now(PARIS_TZ)
    if now < time_to_wait:
        time.sleep((time_to_wait - now).total_seconds())

In [59]:
download_db("https://github.com/belzanne/steampage-creation-date/blob/main/steam_games.db","/Users/juliebelzanne/Documents/Hush_Crasher/steam_data/newgame-newtweet/steam_games.db")

True

In [60]:
connect_to_db("/Users/juliebelzanne/Documents/Hush_Crasher/steam_data/newgame-newtweet/steam_games.db")

In [61]:
get_github_file_content(GITHUB_FILE_PATH)

b''

In [62]:
load_tweeted_games()

Le fichier tweeted_games.json est vide ou n'est pas un JSON valide.


set()

In [63]:
def main():
    # URL de la base de données sur GitHub 
    db_url = "https://raw.githubusercontent.com/votre-username/votre-repo/main/steam_games.db"
    
    tweeted_games = load_tweeted_games()
    
    with tempfile.NamedTemporaryFile(delete=False) as temp_db:
        if download_db(db_url, temp_db.name):
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, tweeted_games)
            
            next_tweet_time = TWEET_START_TIME
            for app_id in new_entries:
                game_data = get_game_details(app_id)
                if game_data and filter_game(game_data):
                    message = create_tweet_message(game_data)
                    
                    wait_until(next_tweet_time)
                    tweet_id = tweet_message(message)
                    
                    if tweet_id:
                        tweeted_games.add(app_id)
                        print(f"Tweet publié pour {game_data['name']} (ID: {tweet_id}) à {next_tweet_time}")
                        save_tweeted_games(tweeted_games)
                        next_tweet_time += TWEET_INTERVAL
            
            conn.close()
        else:
            print("Échec du téléchargement de la base de données")
    
    os.unlink(temp_db.name)  # Supprime le fichier temporaire



In [64]:
if __name__ == "__main__":
    main()

Le fichier tweeted_games.json est vide ou n'est pas un JSON valide.
Échec du téléchargement de la base de données
